# Upload 
> Notes on Writing and Flashing firmware using an IPYNB

- prettify: true

<img src='/images/covers/upload1.webp'>

<details>
<summary>

## Optional - Setup Localhost on Colabs

</summary>

Run this in the local terminal:

```
jupyter notebook \ \
  --NotebookApp.allow_origin='https://colab.research.google.com' \ \
  --port=8888 \ \
  --NotebookApp.port_retries=0 \ \
```

'Then hit the CONNECT dropdown on the top right of this screen to Connect to the Localhost 

</details>
<details open>
<summary>

## Upload Function

</summary>

This is the upload script:

In [60]:
import subprocess
import time
import os
from IPython.display import clear_output 

def upload_code(code): 
    # Save the code to a file 
    with open('./src/main.cpp', 'w') as f:
        f.write(code) 

    # Compile the code
    cmd = "C:\\Users\\charl\\.platformio\\penv\\Scripts\\platformio.exe run --target upload"
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, universal_newlines=True)
    timeout = time.time() + 60*2  # 2 minutes from now

    # Process output
    while True:
        if time.time() > timeout:
            print("Timeout reached")
            process.kill()
            break

        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            clear_output() 
            break
        if output.strip().startswith("Writing at"): 
            clear_output() 
        print(output.strip())

    # Get the output and errors after the process has completed
    output, error = process.communicate()
    # print(output)
    if error:
        print("Error:", error)

def read_and_upload_code(file_path): 
    with open(file_path, 'r') as f:
        code = f.read() 
        upload_code(code)


</details>
<details open>
<summary>

## Uploading

</summary>

This is the uplaod fn in use:

In [61]:
import os
directory = os.path.join(os.getcwd(), 'samples')
files = os.listdir(directory)
for file in files:
    print(file)
# file = 'a_FastLedFTT.cpp'
# file = 'analogSoundSampleSketch.cpp'
# file = 'gpt4_v4.cpp' 
# file = 'blink.cpp'

a_FastLedFTT.cpp
a_mic_2_envelope_analyzed.cpp
a_sound2light3.cpp
a_theatersound.cpp
blink_monitor.cpp
blink_serial.cpp
_analogAudio.cpp
_digitalAudio.cpp
_fft_v1.cpp
_fft_v2.cpp


In [62]:
directory = os.path.join(os.getcwd(), 'lightstrip') 
file = 'main.cpp'

In [63]:
read_and_upload_code(directory+'/'+file)

</details>
<details open>
<summary>

## Serial Logging

</summary>

This is a simple example to read the serial output

In [2]:
import serial
import time
from IPython.display import clear_output

ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM11'
ser.timeout = 1  # Set a short read timeout, e.g., 1 second
ser.open()
ser.flushInput()
timeout = 30

start_time = time.time()  # Record the start time

try:
    while True:
        current_time = time.time()
        if current_time - start_time > timeout: 
            print("Time elapsed. Exiting...")
            break

        inpt = ser.readline()
        if inpt:
            print(inpt.decode('utf-8').strip())
except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally: 
    ser.close()  # Ensure the serial connection is closed
    clear_output()


</details>
<details open>
<summary>

## Serial Plotting

</summary>

this is a more complex example

In [65]:
#collapse_input #hide_output
monitor = """
import serial
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import csv
import time

ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM11'
ser.open()
ser.flushInput()

# Create an empty list to store the data
data = [0] * 100

# Create the plot
plt.ion() # Turn on interactive mode
fig, ax = plt.subplots()
line, = ax.plot(data)
ax.set_title("Random Data")
ax.set_xlabel("Data Point")
ax.set_ylabel("Value")

try:
    while True:
        # Read a line of data from the serial port
        inpt = ser.readline().decode('utf-8').strip()

        # Parse the data and append it to the list
        try:
            value = int(inpt)
            data.pop(0)  # Remove the first value from the list
            data.append(value)  # Append the new value to the end of the list
        except IndexError:
            pass

        # Update the plot
        line.set_ydata(data)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()

        # Display the plot
        clear_output(wait=True)
        display(fig)

        # save the output 
        with open("test_data.csv","a", newline="") as f:
            writer = csv.writer(f,delimiter=",")
            writer.writerow([time.time(),value])

except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally:
    ser.close()
    clear_output()
"""

This could be done in js using the [Web Serial API](https://developer.mozilla.org/en-US/docs/Web/API/Web_Serial_API) too

In [66]:
import os

def list_ino_files(root_dir):
    ino_files = []
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            ino_file = os.path.join(folder_path, folder + '.ino')
            if os.path.isfile(ino_file):
                ino_files.append(ino_file)
    return ino_files

def print_ino_file_contents(file_index, ino_files):
    if 0 <= file_index < len(ino_files):
        with open(ino_files[file_index], 'r') as file:
            contents = file.read()
            print(ino_files[file_index])
            print(contents)
    else:
        print("Invalid index.")

# Change this path to your root directory
root_dir = r"C:\Users\charl\OneDrive\Desktop\code\Arduino\scripts"

# Listing and printing files with indexes
ino_files = list_ino_files(root_dir)
for index, file in enumerate(ino_files):
    relative_path = os.path.relpath(file, root_dir)
    print(f"Index: {index}, File: {relative_path}")

Index: 0, File: analogSoundSampleSketch\analogSoundSampleSketch.ino
Index: 1, File: buttoncycler\buttoncycler.ino
Index: 2, File: espblink\espblink.ino
Index: 3, File: march_25_2023_circuitplayground_fun\march_25_2023_circuitplayground_fun.ino
Index: 4, File: microphone_test_2\microphone_test_2.ino
Index: 5, File: microphone_test_3\microphone_test_3.ino
Index: 6, File: simple\simple.ino
Index: 7, File: simpleblinkesp32\simpleblinkesp32.ino
Index: 8, File: sketch_dec18a\sketch_dec18a.ino
Index: 9, File: sketch_feb10a\sketch_feb10a.ino
Index: 10, File: sketch_feb10b\sketch_feb10b.ino
Index: 11, File: sketch_feb10c_original_simple\sketch_feb10c_original_simple.ino
Index: 12, File: sketch_feb10_StrandtestBLE_nodelay\sketch_feb10_StrandtestBLE_nodelay.ino
Index: 13, File: sketch_jul07a\sketch_jul07a.ino
Index: 14, File: sketch_jul07b\sketch_jul07b.ino
Index: 15, File: sketch_jul07c\sketch_jul07c.ino
Index: 16, File: sketch_jul07f\sketch_jul07f.ino
Index: 17, File: sketch_jul07g\sketch_jul07

In [67]:
# Example usage
# print_ino_file_contents(28, ino_files)  # Replace 0 with your chosen index

</details>